# **Formula AI Hackathon Weather Prediction using Neural Nets - Challenge#1**

# Import dependencies

In [1]:
#import dependencies
import numpy as np
import pandas as pd
import os
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
#from imblearn.over_sampling import SMOTE
#!pip install fancyimpute
#from fancyimpute import IterativeImputer

from sklearn.mixture import GaussianMixture
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns',None)
from sklearn.metrics import accuracy_score,f1_score,recall_score,precision_score,confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
# from sklearn.inspection import permutation_importance

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

### Load dataset

In [3]:
# load dataset
weatherdata = pd.read_csv('/kaggle/input/formulaaihackathon2022/weather.csv')

#copy of wetherdata
df = weatherdata.copy()

## Auto EDA using sweetviz

In [5]:
!pip install sweetviz
import sweetviz as sv

feature_config = sv.FeatureConfig(force_num=['M_PACKET_FORMAT', 'M_GAME_MAJOR_VERSION', 'M_GAME_MINOR_VERSION',
       'M_PACKET_VERSION', 'M_PACKET_ID', 'M_SESSION_UID', 'M_SESSION_TIME',
       'M_FRAME_IDENTIFIER', 'M_PLAYER_CAR_INDEX',
       'M_SECONDARY_PLAYER_CAR_INDEX', 'M_BRAKING_ASSIST',
       'M_SESSION_LINK_IDENTIFIER', 'M_PIT_RELEASE_ASSIST', 'TIMESTAMP',
       'M_ZONE_START', 'M_ZONE_FLAG', 'M_PIT_STOP_WINDOW_IDEAL_LAP',
       'M_TRACK_TEMPERATURE', 'M_TRACK_LENGTH', 'M_GAME_PAUSED',
       'M_FORECAST_ACCURACY', 'M_AIR_TEMPERATURE',
       'M_NUM_WEATHER_FORECAST_SAMPLES', 'M_SLI_PRO_NATIVE_SUPPORT',
       'M_SAFETY_CAR_STATUS', 'M_TRACK_ID', 'M_ERSASSIST', 'M_FORMULA',
       'M_SEASON_LINK_IDENTIFIER', 'M_PIT_ASSIST', 'M_GEARBOX_ASSIST',
       'M_SESSION_TYPE', 'M_SPECTATOR_CAR_INDEX',
       'M_PIT_STOP_WINDOW_LATEST_LAP', 'M_WEEKEND_LINK_IDENTIFIER',
       'M_DYNAMIC_RACING_LINE_TYPE', 'M_SESSION_TIME_LEFT',
       'M_SESSION_DURATION', 'M_PIT_STOP_REJOIN_POSITION',
       'M_WEATHER_FORECAST_SAMPLES_M_SESSION_TYPE', 'M_TIME_OFFSET',
       'M_WEATHER_FORECAST_SAMPLES_M_WEATHER',
       'M_WEATHER_FORECAST_SAMPLES_M_TRACK_TEMPERATURE',
       'M_TRACK_TEMPERATURE_CHANGE',
       'M_WEATHER_FORECAST_SAMPLES_M_AIR_TEMPERATURE',
       'M_AIR_TEMPERATURE_CHANGE', 'M_RAIN_PERCENTAGE', 'M_WEATHER',
       'M_AI_DIFFICULTY', 'M_PIT_SPEED_LIMIT', 'M_NETWORK_GAME',
       'M_TOTAL_LAPS', 'M_STEERING_ASSIST', 'M_IS_SPECTATING',
       'M_DYNAMIC_RACING_LINE', 'M_DRSASSIST', 'M_NUM_MARSHAL_ZONES'])

In [6]:
report = sv.analyze(weatherdata, target_feat='M_WEATHER', feat_cfg=feature_config)

In [7]:
#report.show_html('weather_report.html', open_browser=True)
report.show_notebook()

### Based on the EDA using Sweet Viz, the below columns can be removed due to variance, outliers, either irrelevant or have bad correlation with the Target variable (M_WEATHER)
​
- **M_PACKET_FORMAT**                | {Values, %} = {(2021, 100%)}
- **M_GAME_MAJOR_VERSION**          | { Values, %} = {(1, 100%)}
- **M_GAME_MINOR_VERSION**          | {Values, %} = {(14, 89.3%), (15,10.7%)} -> Irrelavant & no strong correlation to Target feature and strongly biased data.
- **M_PACKET_VERSION**              | {Values, %} = {(1, 100%)}
- **M_PACKET_ID**                    | {Values, %} = {(1, 100%)}
- **M_FRAME_IDENTIFIER**              |  # Identifier for the frame the data was retrieved on -> Irrelavant & no strong correlation to Target feature.
- **M_SECONDARY_PLAYER_CAR_INDEX**  | {Values, %} = {(255, 100%)}
- **M_BRAKING_ASSIST**              | {Values, %} = {(0, 73.4%), (3,16.1%), (2,10.5%)} - **Car Braking assistance -> Irrelavant & no strong correlation to Target feature.
- **M_SESSION_LINK_IDENTIFIER**        | # Identifier for session - **persists across saves - **Dropping this because we already have Session_UID as the identifier for each session.
- **M_PIT_RELEASE_ASSIST**           | {Values, %} = {(0, 68.5%), (1,31.5%)} -> Irrelavant & no strong correlation to Target feature and strongly biased data.
- **M_ZONE_START**                  | # Fraction (0..1) of way through the lap the marshal zone starts -> Irrelavant & no strong correlation to Target feature.
- **M_ZONE_FLAG**                    | # Related to Marshal zones and -> Irrelavant & no strong correlation to Target feature.
- **M_GAME_PAUSED**                    | # Related to user action -> Irrelavant & no strong correlation to Target feature.
- **M_SLI_PRO_NATIVE_SUPPORT**       | {Values, %} = {(0, 100%)} -> Irrelavant & no strong correlation to Target feature.
- **M_SAFETY_CAR_STATUS**            | {Values, %} = {(0, 100%)} -> Irrelavant & no strong correlation to Target feature.
- **M_ERSASSIST**                    | {Values, %} = {(0, 68.5%), (1,31.5%)} -> Irrelavant & no strong correlation to Target feature and strongly biased data.
- **M_FORMULA**                        | {Values, %} =  {(0, 99.7%), (3,0.3%)} # Game Name -> Irrelavant & no strong correlation to Target feature and strongly biased data.
- **M_PIT_ASSIST**                  | {Values, %} = {(0, 68.5%), (1,31.5%)} -> Irrelavant & no strong correlation to Target feature and strongly biased data.
- **M_GEARBOX_ASSIST**              | {Values, %} = {(0, 68.5%), (1,31.5%)} -> Irrelavant & no strong correlation to Target feature and strongly biased data.
- **M_SPECTATOR_CAR_INDEX**         | # Index of the car being spectated -> Irrelavant & no strong correlation to Target feature.
- **M_DYNAMIC_RACING_LINE_TYPE**      | {Values, %} = {(0, 68.5%), (1,31.5%)} -> Irrelavant & no strong correlation to Target feature and strongly biased data.
- **M_PIT_SPEED_LIMIT**                | {Values, %} = {(80, 79.2%), (60,20.8%)} -> Fixed Speed Limits set for PIT -> Irrelavant & no strong correlation to Target feature and strongly biased data.
- **M_NETWORK_GAME**                  | Depends on the gamer/user if game is offline or online (0 or 1) -> Irrelavant & no strong correlation to Target feature and strongly biased data.
- **M_STEERING_ASSIST**       | {Values, %} = {(0, 84.2%), (1,15.8%)} -> Irrelavant & no strong correlation to Target feature and strongly biased data.
- **M_IS_SPECTATING**       | 99.9% zero values -> Irrelavant & no strong correlation to Target feature and strongly biased data.
- **M_DYNAMIC_RACING_LINE**     | {Values, %} = {(0, 68.5%), (1,31.5%)} -> Irrelavant & no strong correlation to Target feature and strongly biased data.
- **M_DRSASSIST**             | {Values, %} = {(0, 68.5%), (1,31.5%)} -> Irrelavant & no strong correlation to Target feature and strongly biased data.
- **M_NUM_MARSHAL_ZONES**     | # Number of marshal zones to follow -> Irrelavant & no strong correlation to Target feature.
- **GAMEHOST**               | Only 2 values and Irrelavant to weather prediction. Strongly biased data.
- **'Unnamed: 58'**             | 100% Totally EMPTY column

# EDA and Preprocessing

In [3]:
#drop columns(either based on variance or outliers)
irrelavant_features = ['M_PACKET_FORMAT', 'M_GAME_MAJOR_VERSION', 'M_GAME_MINOR_VERSION', 'M_PACKET_VERSION', 
                       'M_PACKET_ID', 'M_FRAME_IDENTIFIER', 'M_SECONDARY_PLAYER_CAR_INDEX', 'M_BRAKING_ASSIST', 'M_SESSION_LINK_IDENTIFIER', 
                       'M_PIT_RELEASE_ASSIST', 'M_ZONE_START', 'M_ZONE_FLAG', 'M_GAME_PAUSED', 'M_SLI_PRO_NATIVE_SUPPORT', 
                       'M_SAFETY_CAR_STATUS', 'M_ERSASSIST','M_FORMULA', 'M_PIT_ASSIST', 'M_GEARBOX_ASSIST', 'M_SPECTATOR_CAR_INDEX', 'M_DYNAMIC_RACING_LINE_TYPE', 
                       'M_PIT_SPEED_LIMIT', 'M_NETWORK_GAME', 'M_STEERING_ASSIST', 'M_IS_SPECTATING', 'M_DYNAMIC_RACING_LINE', 
                       'M_DRSASSIST', 'M_NUM_MARSHAL_ZONES','GAMEHOST','Unnamed: 58']

df.drop(columns=irrelavant_features,inplace=True)

#split the feats into discrete and continous
cont_feats = [feat for feat in df.columns if df[feat].nunique()>50]
cat_feats = [feat for feat in df.columns if feat not in cont_feats]

### Data Wrangling

In [9]:
def missing_values_table(df):
        zero_val = (df == 0.00).astype(int).sum(axis=0)
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mz_table = pd.concat([zero_val, mis_val, mis_val_percent], axis=1)
        mz_table = mz_table.rename(
        columns = {0 : 'Zero_Values', 1 : 'Missing_Values', 2 : 'Missing_Values%'})
        mz_table['Sum_Zero_Plus_Missing_Values'] = mz_table['Zero_Values'] + mz_table['Missing_Values']
        mz_table['%_Sum_Zero_Plus_Missing_Values'] = 100 * mz_table['Sum_Zero_Plus_Missing_Values'] / len(df)
        mz_table['Data Type'] = df.dtypes
        mz_table = mz_table[
            mz_table.iloc[:,1] != 0].sort_values(
        'Missing_Values%', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns and " + str(df.shape[0]) + " Rows.\n"      
            "There are " + str(mz_table.shape[0]) +
              " columns that have missing values.")
#         mz_table.to_excel('D:/sampledata/missing_and_zero_values.xlsx', freeze_panes=(1,0), index = False)
        return mz_table

missing_values_df = missing_values_table(df)
missing_values_df

### Since we have 27.3% of missing values in 8 Features, we shall use Forward Fill to Fill the values. The reason for using forward fill is due to heavy existence of duplicates.

In [4]:
#fill null values with ffill or agregated mean by date and time
df = df.fillna(method = 'ffill')

In [5]:
df['TIMESTAMP'] = df['TIMESTAMP'].apply(lambda x:datetime.fromtimestamp(x))
df['date'] =  df['TIMESTAMP'].dt.date
df['hour'] =  df['TIMESTAMP'].dt.hour
df['min'] =  df['TIMESTAMP'].dt.minute
df['sec'] =  df['TIMESTAMP'].dt.second

#sort values by date
df.sort_values(by=['TIMESTAMP'],inplace=True)

In [7]:
#copy of df 
dfcopy = df.copy()

#aggregate df by  M_SESSION_UID - M_SESSION_TIME - M_TIME_OFFSET
df_agg = df.groupby(['M_SESSION_UID','M_SESSION_TIME','M_TIME_OFFSET']).mean().reset_index()

In [8]:
df_agg

## Using Random Forest Classifier to predict the weather accuracy using the Target 'M_WEATHER'

In [9]:
target = 'M_WEATHER'
# df_agg = df_agg.drop(columns=['date'])

X = df_agg.drop(columns=target)
y = df_agg[target]
X_train, X_test, y_train, y_test = train_test_split(X, y,shuffle=False,random_state=42)
#random forest classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

In [10]:
#predict for train and validation data
pred_train = rf.predict(X_train)
pred_test = rf.predict(X_test)

In [12]:
def metrics_print(actual,predicted, data_set):
    print(f'{data_set}')
    print('ACCURACY',accuracy_score(actual,predicted))
    print('RECALL', recall_score(actual,predicted,average='weighted'))
    print('PRECISION',precision_score(actual,predicted,average='weighted'))
    print('F1-SCORE',f1_score(actual,predicted,average='weighted'))

In [13]:
metrics_print(y_train,pred_train,'train')
metrics_print(y_test,pred_test,'test')

- The accuracy of the Target Variable 'M_WEATHER' is 83.2% which is good.

In [14]:
discrete_feats = [col for col in df_agg.columns if df_agg[col].nunique() <25]
cont_feats = [col for col in df_agg.columns if col not in discrete_feats]

In [26]:
#discrete features
f, ax = plt.subplots(6, 3, figsize = (35, 40))
axx = ax.flatten()

for index, col in enumerate(discrete_feats):
    df_agg[ col].value_counts().plot.pie(ax = axx[index])

In [29]:
#bivariate analysis of categorical features with target
f, ax = plt.subplots(9, 2, figsize = (40 , 40))
axx = ax.flatten()

for index, col in enumerate(discrete_feats):
    sns.countplot(data = df_agg, x = col, hue = target, ax = axx[index])

In [30]:
#bivariate analysis of categorical features with target
f, ax = plt.subplots(5, 3, figsize = (20,20))
axx = ax.flatten()
for index, col in enumerate(cont_feats):
#     plt.figure(figsize=(15,8))
    sns.distplot(df_agg[col],ax=axx[index])
#     plt.show()

## Feature Engineering

In [21]:
#mean TRACK TEMP AND AIR TEMP per session id and hour
feats_gr  = ['M_SESSION_UID','hour']
feats_agg = ['M_TRACK_TEMPERATURE','M_AIR_TEMPERATURE']
dftemp = df_agg.groupby(feats_gr)[feats_agg].agg([np.mean])
cols = dftemp.columns
dftemp.columns = ['_SessionID_Hour_'.join(c) for c in cols]
dftemp.reset_index(inplace = True)
# join df with df_agg
df_agg = df_agg.merge(dftemp, on = feats_gr, how = 'left')

#drop features by variance
dropfeats = ['M_PIT_STOP_WINDOW_IDEAL_LAP','M_FORECAST_ACCURACY']

In [18]:
# #session features 
sess_feats = ['M_SESSION_UID','M_SESSION_TYPE','M_SESSION_TIME','M_SESSION_TIME_LEFT','M_SESSION_DURATION']

### Gaussian Mixture Model for Target Recalibration

In [19]:
df.M_WEATHER.value_counts()

### Based on the value counts of Target Variable 'M_WEATHER', there are missing values [3,4] which would cause a misleading machine learning prediction.

- **To overcome such a crisis, we re-calibrate the Target Variable by using GMM Classification Model and create a new Target Variable 'WEATHER_CLASS'**
- **Top 3 highly correlavant features are taken and GMM Classification Model is ran on them to get the GMM_CLASS with 6 WEATHER Classes.**

In [22]:
#get high correlated features for GMM model
highcorrelationfeats = ['M_AIR_TEMPERATURE', 'M_TRACK_TEMPERATURE_SessionID_Hour_mean','M_TRACK_TEMPERATURE']
tempdf = df_agg[highcorrelationfeats]

#Fit data using GMM
gmm = GaussianMixture(n_components=6, random_state = 1)
gmm.fit(tempdf)
tempdf['GMM_CLASS'] = gmm.predict(tempdf)


In [23]:
tempdf

In [24]:
tempdf['GMM_CLASS'].value_counts()

In [25]:
tempdf.groupby(['GMM_CLASS'])['M_TRACK_TEMPERATURE'].median().sort_values(ascending=False)

- **GMM_CLASS values are created with 6 values that are highly correlavant to the top 3 features ['M_AIR_TEMPERATURE', 'M_TRACK_TEMPERATURE_SessionID_Hour_mean','M_TRACK_TEMPERATURE']**

In [26]:
fig = plt.figure(figsize = (5,5))
sns.set(style = 'whitegrid')

sns.barplot(tempdf.GMM_CLASS, tempdf.M_TRACK_TEMPERATURE)

In [27]:
fig = plt.figure(figsize = (5,5))
sns.set(style = 'whitegrid')

sns.barplot(df_agg.M_WEATHER, df_agg.M_TRACK_TEMPERATURE)

In [28]:
 for feat in tempdf.columns:
     sns.displot(data=tempdf, x=feat, hue=df_agg['M_WEATHER'],
                 kind='kde',palette=sns.color_palette('bright')[:4],height=4,aspect=2.2)
     plt.show()

### We compare the existing weather density distribution based on the highly correlated features to map the GMM_CLASS values.

In [29]:
#map gmm class
gmmlabel = list(tempdf.groupby(['GMM_CLASS'])['M_TRACK_TEMPERATURE'].median().sort_values(ascending=False).index.values)
print('GMM_CLASS:', gmmlabel)

weatherlabel = [0,1,2,3,4,5]
print('M_WEATHER:',weatherlabel)

In [30]:
#map function to map GMM_CLASS to M_WEATHER
gmm_map = dict(zip(gmmlabel,weatherlabel))
print(gmm_map)

In [31]:
#map gmm class
df_agg['WEATHER_CLASS'] = tempdf['GMM_CLASS'].map(gmm_map)

In [32]:
sess_feats

In [33]:
df_agg['WEATHER_CLASS'].value_counts()

### Random Forest Classifier to check the accuracy of the new Target Variable 'WEATHER_CLASS'

In [34]:
target = 'WEATHER_CLASS'
# df_agg = df_agg.drop(columns=['date'])

X = df_agg.drop(columns=['WEATHER_CLASS','M_WEATHER'])
y = df_agg[target]
X_train, X_test, y_train, y_test = train_test_split(X, y,shuffle=False,random_state=786)
#random forest classifier
rf = RandomForestClassifier(n_estimators=100, random_state=786)
rf.fit(X_train, y_train)

In [35]:
#predict for train and validation data
pred_train = rf.predict(X_train)
pred_test = rf.predict(X_test)

metrics_print(y_train,pred_train,'train')
metrics_print(y_test,pred_test,'test')

- **The accuracy of new Target Variable 'WEATHER_CLASS' is 85% and is better than the original target variable 'M_WEATHER' which is 83.2%.**

In [36]:
con_df = pd.DataFrame(list(zip(y_test.values,pred_test)),columns=['y_Actual','y_Predicted'])
confusion_matrix = pd.crosstab(con_df['y_Actual'], con_df['y_Predicted'], rownames=['Actual'], colnames=['Predicted'])
print (confusion_matrix)

## Predict Weather using TabNet Classifier Algorithm

In [37]:
from sklearn.metrics import accuracy_score

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

!pip install pytorch-tabnet
from pytorch_tabnet.tab_model import TabNetClassifier

In [38]:
cols = df_agg.columns.to_list()
cols.remove('WEATHER_CLASS')
cols.remove('M_WEATHER')
cols

In [39]:
finaldf = df_agg.copy()
finaldf.drop(columns=['M_WEATHER'], inplace = True)

In [40]:
finaldf.columns

In [41]:
finaldf

### Split the data into train and test data sets with a 70:30 ratio

In [42]:
from sklearn.model_selection import train_test_split

training_data, testing_data = train_test_split(finaldf, test_size=0.3, random_state=25, shuffle = False)

print(f"No. of training examples: {training_data.shape}")
print(f"No. of testing examples: {testing_data.shape}")

### Drop Target Variable 'WEATHER_CLASS' for test data

In [43]:
test = testing_data.copy()
test.drop(columns=['WEATHER_CLASS'], inplace = True)

In [44]:
test.shape

In [45]:
data = training_data.copy()
data.shape

In [46]:
final_pred_df = finaldf.copy()
final_pred_df.drop(columns=['WEATHER_CLASS'], inplace = True)

In [47]:
final_pred_df.shape

## Fit and Tranform the data using the TabNet Classifier Model

In [48]:
N_folds = 5
seeds = [455, 485, 659, 16, 5659]

data_preds = np.zeros((data.shape[0], 6))
test_preds = np.zeros((test.shape[0], 6))
final_preds = np.zeros((final_pred_df.shape[0], 6))

feature_importance = pd.DataFrame()

for seed in seeds:
    print('---------------------------------------')
    print('SEED : ', seed)
    print('---------------------------------------')
    
    
    data['fold'] = -1
    skf = StratifiedKFold(n_splits=N_folds, shuffle=False, random_state=seed)
    for f, (_, idxs) in enumerate(skf.split(data, data['WEATHER_CLASS'])):
        data.loc[idxs, 'fold'] = f
        
    for F in range(N_folds):
        
        print('FOLD : ', F)
        
        train = data[data['fold'] != F].reset_index(drop=True)
        valid = data[data['fold'] == F].reset_index(drop=True)
        
        
        model = TabNetClassifier(verbose=0,seed=42)
        model.fit(X_train=train[cols].values, y_train=train['WEATHER_CLASS'].values,
                  eval_set=[(train[cols].values, train['WEATHER_CLASS'].values), (valid[cols].values, valid['WEATHER_CLASS'].values)],
                  eval_name=['train', 'valid'],
                  patience=5, max_epochs=100,
                  eval_metric=['accuracy'],
                  batch_size=1024, virtual_batch_size=256)
        
        
        preds = model.predict_proba(valid[cols].values)
        data_preds[data[data['fold'] == F].index] += preds
        
        preds = model.predict_proba(test[cols].values)
        test_preds += preds
        
        preds = model.predict_proba(final_pred_df[cols].values)
        final_preds += preds

In [50]:
data_preds1 = data_preds / (len(seeds))
test_preds1 = test_preds / (len(seeds) * N_folds)

In [63]:
final_preds[0]

In [65]:
finalpred_avg= final_preds/ (len(seeds) * N_folds)

In [68]:
finalpred_avg

In [71]:
final_pred_df

In [72]:
finaltestdf = final_pred_df.copy()
#predict the weather class  prob
prob = finalpred_avg
finaltestdf['Weather_Prediction_Class_Probability'] = np.max(prob,axis=1)
#predict  weather class
finaltestdf['Weather_Prediction_Class']  = np.argmax(prob,axis=1)
finaltestdf

# Predict Weather using Multi-Class Classification (TensorFlow)

In [90]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler

#copy of df_agg
dffinal = df_agg.copy()

#split data into target and features
X = df_agg.drop(columns=['M_WEATHER','WEATHER_CLASS'])
Y = df_agg['WEATHER_CLASS']

#scale X using standardscaler
std_scale = StandardScaler()
X_scaled = std_scale.fit_transform(X)
X_df = pd.DataFrame(X_scaled,columns=X.columns)

#train test split 
X_train, X_test, y_train, y_test = train_test_split(X_df, Y,shuffle=False,random_state=42)


NN_model = tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=X.shape[1]),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(6, activation='softmax') ])


NN_model.compile(Adam(lr = 0.01), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


NN_model.fit(X_train,y_train, 
          validation_data=(X_test, y_test), 
          epochs=50, batch_size=10000, verbose=1,
          )

In [74]:
X

In [77]:
#predict the weather class  prob
#prob = finalpred_avg
xnn_df = X.copy()

xnn_df['Weather_Prediction_Class_Probability'] = np.max(NN_model.predict(X))
#predict  weather class
xnn_df['Weather_Prediction_Class']  = np.argmax(NN_model.predict(X))
xnn_df


In [75]:
NN_model.predict(X)

In [82]:
finaltestdf['Ensemble_weather_prob'] = (xnn_df['Weather_Prediction_Class_Probability'] + finaltestdf['Weather_Prediction_Class_Probability'])/2

In [91]:
finaltestdf

In [84]:

finalpreddf = finaltestdf[['M_SESSION_UID','M_SESSION_TIME','M_TIME_OFFSET','Weather_Prediction_Class','Ensemble_weather_prob']]

#merge dfcopy with finalpreddf to get timestamp
df_sessid_time = dfcopy.drop_duplicates(subset=['M_SESSION_UID','TIMESTAMP'])
finaldfpred_with_timestamp = finalpreddf.merge(df_sessid_time[["M_SESSION_UID","M_SESSION_TIME",'TIMESTAMP']], on=["M_SESSION_UID","M_SESSION_TIME"])
finaldfpred_with_timestamp = finaldfpred_with_timestamp.drop(columns=['M_SESSION_TIME'])

#final output
finaldfpred_with_timestamp['Output'] = finaldfpred_with_timestamp.apply(lambda x:{x['M_TIME_OFFSET']:{'type':x['Weather_Prediction_Class'],x['Weather_Prediction_Class']:x['Ensemble_weather_prob']}},axis=1)
finaloutput = finaldfpred_with_timestamp.groupby(['M_SESSION_UID','TIMESTAMP'])['Output'].apply(lambda x:list(x)).reset_index()

In [85]:
finaloutput

In [86]:
finaloutput.to_csv('FINAL_OUTPUT.csv')

In [87]:
NN_model.save('saved_model/my_model')